In [78]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import bs4 as bs
import re

In [79]:
bs.__version__

'4.9.3'

In [80]:
# Cogemos la url de la web El Mundo: Sanidad
url_sanidad = 'https://www.elmundo.es/ciencia-y-salud/salud.html'
url_tecnologia = 'https://www.elmundo.es/tecnologia.html'
url_ciencia = 'https://www.elmundo.es/ciencia-y-salud/ciencia.html'

In [81]:
def getHrefs(page):
    # Hacemos la solicitud de requests a la url
    page_sanidad = requests.get(url_sanidad)
    # Accedemos al contenido especifico de la página
    soup = BeautifulSoup(page_sanidad.text, 'html.parser')
    # Buscamos todos los container a que tengan esa clase, que es la que guarda los href de cada noticia
    containers = soup.find_all('div', {'class':'ue-l-cover-grid__unit ue-l-cover-grid__unit--no-grow'})
    # Lista para almacenar los enlaces
    hrefs = []
    for enlace in containers:
        noticia = enlace.find('a')['href']
        hrefs.append(noticia)
    print(hrefs)

In [82]:
listaURLsElMundo = [url_sanidad, url_tecnologia, url_ciencia]

In [95]:
def getFieldsFromPages():
    cont = 1
    for link in range(len(listaURLsElMundo)):
        # Hacemos la solicitud de requests a la url
        page = requests.get(listaURLsElMundo[link])
        # Accedemos al contenido especifico de la página
        soup = BeautifulSoup(page.text, 'html.parser')
        # Buscamos todos los container a que tengan esa clase, que es la que guarda los href de cada noticia
        containers = soup.find_all('div', {'class':'ue-l-cover-grid__unit ue-l-cover-grid__unit--no-grow'})
        # Lista para almacenar los enlaces
        hrefs = []
        for enlace in containers:
            noticia = enlace.find('a')['href']
            hrefs.append(noticia)
        cont = 1
        for href in range(len(hrefs)):
            # Pillamos cada noticia mediante su href individual
            noticia = requests.get(hrefs[href])
            # Accedemos al contenido especifico de cada noticia
            soupNoticia = BeautifulSoup(noticia.text, 'html.parser')
            # cogemos el container principal que es el tiene todos los fields que nos interesan
            fields = soup.find_all('article', {'class' : 'ue-c-article has-image'})
            # Lista para almacenar los enlaces
            noticias = []
            # Cogemos el titulo
            try: 
                titulo = soupNoticia.find('h1', {'class': 'ue-c-article__headline js-headline'}).text
            except: 
                titulo = "Noticia sin titulo"
            # Cogemos la entradilla
            try: 
                entradilla = soupNoticia.find('p', {'class':'ue-c-article__standfirst'}).text
            except: 
                entradilla = "Noticia sin entradilla"
            # cogemos el field para coger el texto
            try: 
                estructuraTexto = soupNoticia.find('div', {'class': 'ue-l-article__body ue-c-article__body'})
                # Cogemos los p de la estructura de texto que conformaran todo el texto que necesitamos
                childrenTexto = estructuraTexto.findChildren("p" , recursive=False)
                texto=""
                for childTexto in childrenTexto:
                    texto = texto + childTexto.text
            except: 
                texto = "Noticia sin texto"
            # Cogemos las etiquetas
            try:
                estructuraEtiquetas = soupNoticia.find('ul', {'class' : 'ue-c-article__tags'})
                childrenEtiquetas = estructuraEtiquetas.findChildren("li" , recursive=False)
                tags=""
                for childEtiquetas in childrenEtiquetas:
                    tags = tags + childEtiquetas.text + "|"
            except: 
                tags = "Noticia sin etiquetas"
            # Todo el string que meteremos al archivo de texto
            noticiaElMundo = titulo + "\n######\n" + entradilla + "\n######\n" + texto + "\n######\n" + tags + "\n\n"
            #print(noticiaElMundo)
            delDir_v1 = re.sub('.*\/',"", listaURLsElMundo[link])
            delDir = re.sub('.html', "", delDir_v1)
            file = open("./ElMundo/" + delDir + "/ElMundo_" + delDir + str(cont) + ".txt", "w", encoding="utf-8")
            file.write(noticiaElMundo)
            file.close()
            cont=cont+1

In [96]:
noticiasElMundo = getFieldsFromPages()

In [67]:
def getFieldsFromHref(href):
    # Hacemos la solicitud de requests a la url
    page = requests.get(href)
    # Accedemos al contenido especifico de la página
    soup = BeautifulSoup(page.text, 'html.parser')
    # Buscamos todos los container a que tengan esa clase, que es la que guarda los href de cada noticia
    containers = soup.find_all('div', {'class':'ue-l-cover-grid__unit ue-l-cover-grid__unit--no-grow'})
     # Lista para almacenar los enlaces
    hrefs = []
    for enlace in containers:
        noticia = enlace.find('a')['href']
        hrefs.append(noticia)
    for href in range(len(hrefs)):
        # Pillamos cada noticia mediante su href individual
        noticia = requests.get(hrefs[href])
        # Accedemos al contenido especifico de cada noticia
        soupNoticia = BeautifulSoup(noticia.text, 'html.parser')
        # cogemos el container principal que es el tiene todos los fields que nos interesan
        fields = soup.find_all('article', {'class' : 'ue-c-article has-image'})
        # Lista para almacenar los enlaces
        noticias = []
        # Cogemos el titulo
        try: 
            titulo = soupNoticia.find('h1', {'class': 'ue-c-article__headline js-headline'}).text
        except: 
            titulo = "Noticia sin titulo"
        # Cogemos la entradilla
        try: 
            entradilla = soupNoticia.find('p', {'class':'ue-c-article__standfirst'}).text
        except: 
            entradilla = "Noticia sin entradilla"
        # cogemos el field para coger el texto
        try: 
            estructuraTexto = soupNoticia.find('div', {'class': 'ue-l-article__body ue-c-article__body'})
            # Cogemos los p de la estructura de texto que conformaran todo el texto que necesitamos
            childrenTexto = estructuraTexto.findChildren("p" , recursive=False)
            texto=""
            for childTexto in childrenTexto:
                texto = texto + childTexto.text
        except: 
            texto = "Noticia sin texto"
        # Cogemos las etiquetas
        try:
            estructuraEtiquetas = soupNoticia.find('ul', {'class' : 'ue-c-article__tags'})
            childrenEtiquetas = estructuraEtiquetas.findChildren("li" , recursive=False)
            tags=""
            for childEtiquetas in childrenEtiquetas:
                tags = tags + childEtiquetas.text + "|"
        except: 
            tags = "Noticia sin etiquetas"
        # Todo el string que meteremos al archivo de texto
        noticiaElMundo = titulo + "\n######\n" + entradilla + "\n######\n" + texto + "\n######\n" + tags + "\n\n"
        print(noticiaElMundo)
        

In [68]:
getFieldsFromHref(url_sanidad)

¿Y si ómicron fuera la variante que la ciencia estaba esperando? Estos son los escenarios que puede provocar
######
Ómicron, la nueva variante altamente contagiosa, podría tener consecuencias nefastas. Pero si fuera menos agresiva que Delta podría conducir a una coexistencia pacífica con el virus, lo que marcaría efectivamente el final de la pandemia.
######
La llegada de ómicron, la nueva variante de coronavirus identificada por primera vez en Botswana y Sudáfrica y ahora presente en Europa, en Estados Unidos y en muchos otros países del mundo, ha sido recibida con gran ansiedad y miedo. Muchas naciones han cerrado sus fronteras impidiendo el aterrizaje de vuelos desde Sudáfrica y otras regiones africanas, una elección criticada por muchos científicos (y por la OMS) porque la actitud de cierre no anima a los países a comunicar rápidamente las nuevas variantes si luego son castigados con el aislamiento. La Organización Mundial de la Salud identificó inmediatamente a ómicron como una va

Ómicron: lo que sabemos (y lo que no) sobre la variante que más preocupa y sus posibles orígenes
######
Noticia sin entradilla
######
Hace poco más de una semana que conocimos su existencia y todavía sabemos muy pocas cosas sobre ella, pero ómicron, la nueva variante del SARS-CoV-2, ya ha conseguido que todo el planeta esté pendiente de su evolución.Sudáfrica fue, el pasado 25 de n
######
Noticia sin etiquetas


Un informe de Fernando Simón dice que "aumenta la evidencia" de que la variante Ómicron "escapa a la inmunidad por la vacuna"
######
"Se han detectado numerosos casos de ómicron a nivel mundial y aumentan las evidencias de que esta variante escape a la respuesta inmunitaria originada por las variantes previas y por las vacunas", recoge el Ministerio de Sanidad
######
Un informe, firmado por el Centro de Coordinación de Alertas y Emergencias Sanitarias (CCAES), dirigido por el doctor Fernando Simón, advierte de que "aumenta la evidencia" de que la variante ómicron de la Covid "e

¿Por qué se necesitará una vacuna de refuerzo contra el Covid al año? "Porque la enfermedad será endémica"
######
"Las variantes son la peculiaridad de este virus. La inmunidad colectiva es un objetivo inalcanzable", aseguraFabrizio Pregliasco, virólogo de la Universidad de Milán
######
Una vacuna contra el Covid todos los años, durante muchos años. Este es el escenario que plantea Albert Bourla, director general de Pfizer, en una entrevista con la BBC . "Si vamos a hacer una suposición basada en todo lo que he visto hasta ahora, diría que probablemente se necesitarán vacunas anuales para mantener niveles de protección muy robustos y muy, muy altos", dijo.Sin un supuesto "conflicto de intereses", la hipótesis de Albert Bourla no es tan extraña. De hecho, como dice el profesor Fabrizio Pregliasco, virólogo de la Universidad de Milán y director médico de Irccs Galeazzi en Milán, "es obvio, porque el virus permanecerá con nosotros y probablemente tendremos que seguir protegiéndonos de nue

Sanidad insiste en recomendar por el riesgo de ómicron que se limiten los participantes en las reuniones navideñas
######
La actualización de riesgos difundida este viernes reconoce que "se desconoce la efectividad vacunal" frente a la variante ómicron del coronavirus
######
El Centro de Coordinación de Alertas y Emergencias Sanitarias (CCAES) ha actualizado este viernes, por segunda vez en esta semana y cuando arranca el puente de diciembre, su 'Evaluación rápida de riesgo' de la variante ómicron del coronavirus. El organismo dependiente de Sanidad justifica el nuevo documento porque en el contexto intrnacional "aumentan las evidencias de que esta variante escape a la respuesta inmunitaria originada por las variantes previas y por las vacunas". Tras analizar el número de casos confirmados en el mundo y su progresiva aparición por diferentes partes del planeta, en el apartado referido a España esa incógnita se apunta la misma idea, pero con una fórmula más matizada: "Se desconoce la ef

La incidencia sube otros 14 puntos hasta situarse en los 248 casos por 100.000 habitantes
######
Sanidad también notifica 13.738 contagios y 37 muertes 
######
Las comunidades autónomas han notificado este viernes al Ministerio de Sanidad 13.738 nuevos casos de Covid-19, 8.119 de ellos diagnosticados en las últimas 24 horas. La cifra total de contagios en España se eleva ya a 5.202.958 desde el inicio de la pandemia, según las estadísticas oficiales.La incidencia acumulada en los últimos 14 días por 100.000 habitantes se sitúa en 248,18 frente a 234,03 del jueves. En las pasadas dos semanas se ha registrado un total de 117.762 positivos. En el informe de este viernes se han añadido 37 nuevos fallecimientos. Hasta 88.159 personas con prueba diagnóstica positiva han fallecido desde que el virus llegó a España, de acuerdo con los datos recogidos por el Ministerio.La situación en los hospitales españoles empeora desde el jueves. Actualmente, hay 4.262 pacientes ingresados por COVID-19 en t

...

EEUU confirma el primer caso de ómicron en California
######
El portador de la variante del coronavirus viajó desde Sudáfrica el pasado 22 de noviembre
######
Era cuestión de tiempo, como ya habían advertido los expertos. La variante ómicron del Covid-19 ha sido detectada en Estados Unidos, concretamente en California, según confirmaron hoy las autoridades sanitarias estadounidenses. La persona en cuestión, cuya identidad no se ha hecho pública, viajó desde Sudáfrica hasta California el pasado 22 de noviembre, estaba vacunada y padece de síntomas leves que han ido mejorando en los últimos días. EEUU se suma así a una lista de más de una docena de países -España, entre ellos-, en los que se ha detectado la presencia de esta variante que la Organización Mundial de la Salud considera "muy seria" y que se ha extendido por cuatro continentes desde que fuera identificada en Sudáfrica. La confirmación del primer caso coincide con los planes de los Centros de Control y Prevención de Enfermedad

España alcanza los 217 casos tras subir casi nueve puntos en 24 horas
######
Sanidad también notifica 10.536 contagios y 28 muertes
######
Las comunidades autónomas han notificado este miércoles al Ministerio de Sanidad 10.536 nuevos casos de Covid-19, 6.836 de ellos diagnosticados en las últimas 24 horas. La cifra total de contagios en España se eleva ya a 5.174.720 desde el inicio de la pandemia, según las estadísticas oficiales.La incidencia acumulada en los últimos 14 días por 100.000 habitantes se sitúa en 217,1 frente a 208,05 del martes. En las pasadas dos semanas se ha registrado un total de 103.018 positivos. En el informe de este miércoles se han añadido 28 nuevos fallecimientos. Hasta 88.080 personas con prueba diagnóstica positiva han fallecido desde que el virus llegó a España, de acuerdo con los datos recogidos por el Ministerio.La situación en los hospitales españoles empeora desde el martes. Actualmente, hay 3.957 pacientes ingresados por COVID-19 en toda España (3.847 

Quién va a recibir la tercera dosis, cuándo y con qué vacuna contra el coronavirus
######
A la dosis adicional de los grupos más vulnerables, se suma desde el 25 de octubre un 'pinchazo' de refuerzo para los mayores de 70 años
######
La Agencia Europea del Medicamento (EMA) avala el uso de las dosis adicionales y las de refuerzo. En España ya se están poniendo las primeras y se ha dado luz verde a las segundas.Sanidad quiere que los 'pinchazos' de refuerzo estén listos antes de que finalice 2021. En total, esto afecta a 16 millones de españoles.Las dosis que se han aprobado por el Ministerio de Sanidad son adicionales. Esto es, que sirven para completar una pauta vacunal que puede ser de varias dosis, las necesarias para generar una respuesta inmune duradera en el organismo de la persona.Las dosis de refuerzo o booster son aquellas que se administran pasado un tiempo de la primera dosis o pauta vacunal para hacer que las defensas del organismo recuerden cómo actuar ante el virus. Deben

Sanidad aconseja limitar asistentes a eventos sociales, sobre todo en Navidad
######
El Ministerio de Sanidad pide también intensificar las medidas de control y seguimiento de las personas procedentes del sur de África
######
El Ministerio de Sanidad ha recomendado limitar el número de participantes en los "eventos públicos y sociales", especialmente durante las celebraciones de las fiestas navideñas, así como intensificar las medidas de control y seguimiento de las personas procedentes del sur de África.Así lo aconseja el Centro de Coordinación de Alertas y Emergencias Sanitarias (CCAES) en la sexta actualización del documento en el que evalúa el riesgo de las variantes del Sars-CoV-2, en la que señala que "el riesgo de nuevas introducciones de ómicron en España se considera muy alto y el de diseminación, alto". "Debido al contexto epidemiológico actual y a las incertidumbres asociadas a la variante ómicron, así como una posible disminución de la efectividad vacunal con el tiempo, se 

Reino Unido anuncia que ofrecerá la tercera dosis a todos los adultos antes de que termine enero
######
El Gobierno británico quiere potenciar la vacunación ante la llegada de ómicron, que podría estar transmitiéndose ya a nivel comunitario según reconocen las autoridades británicas, aunque descartan un nuevo confinamiento.
######
Los datos todavía no son alarmantes, pero Reino Unido no esconde que la incidencia de la variante ómicron del coronavirus podría ir más allá de los 22 casos que han sido confirmados hasta el día de hoy. "Hay que ser realistas, es probable que se esté produciendo una transmisión comunitaria", ha admitido Sajid Javid, ministro de Sanidad británico. Es, por tanto, según reconocen las autoridades sanitarias del país, cuestión de tiempo que la ómicron se extienda por todo el territorio, por lo que lo relevante ahora es saber qué hacer durante el margen que se obtenga con las nuevas restricciones. Por el momento, y descartada ya la posibilidad de volver a un nuevo 

In [69]:
getFieldsFromHref(url_ciencia)

Volcán La Palma, última hora en directo | Surge una nueva boca al oeste del cementerio de Las Manchas
######
Se cumplen 83 días desde el inicio de la emergencia por el volcán de La Palma y 76 del comienzo de la erupción
######
EL Instituto Volcanológico de Canarias (Involcán) ha detectado la aparición de una nueva boca del volcán de La Palma al oeste del cementerio de Las Manchas, sepultado la semana pasada por la lava. Esta nueva boca de volcán está siendo analizada por los expertos para poder determinar sus posibles consecuencias en las edificaciones que se encuentran cercanas.
######
Volcán La Palma|La Palma|Ciencia y Salud|ciencia|Canarias|España|


Festival de planetas y dos lluvias de estrellas para animar el cielo en diciembre
######
Noticia sin entradilla
######
Diciembre nos ofrece oportunidades magníficas para observar a nuestros vecinos del sistema solar. Venus, Saturno y Júpiter brillan esplendorosamente en los atardeceres, mientras que Marte comienza a verse al alba. Dos l

Volcán La Palma, última hora en directo | Surge una nueva boca al oeste del cementerio de Las Manchas
######
Se cumplen 83 días desde el inicio de la emergencia por el volcán de La Palma y 76 del comienzo de la erupción
######
EL Instituto Volcanológico de Canarias (Involcán) ha detectado la aparición de una nueva boca del volcán de La Palma al oeste del cementerio de Las Manchas, sepultado la semana pasada por la lava. Esta nueva boca de volcán está siendo analizada por los expertos para poder determinar sus posibles consecuencias en las edificaciones que se encuentran cercanas.
######
Volcán La Palma|La Palma|Ciencia y Salud|ciencia|Canarias|España|


Noticia sin titulo
######
Noticia sin entradilla
######
Noticia sin texto
######
Noticia sin etiquetas


Identifican en Tanzania las huellas de un homínido desconocido que vivió hace 3,6 millones de años
######
Un nuevo análisis de unas misteriosas huellas halladas en Laetoli en los 70 apuntan a que pertenecían a una especie de homínido

Despega la primera nave espacial de la NASA que intentará desviar un asteroide
######
Se conocen 10.000 asteroides cercanos a la Tierra de un tamaño de 140 metros o más, pero ninguno tiene una posibilidad significativa de impactar en los próximos 100 años
######
La NASA ha lanzado a primera hora de esta mañana su misión DART con el objetivo de estrellar deliberadamente una nave espacial contra un asteroide, un ensayo en caso de que la humanidad necesite un día impedir que una roca espacial gigante acabe con la vida en la Tierra.Retransmitido en directo por la televisión de la NASA, un cohete SpaceX que cargaba con la sonda experimental despegó a las 22H21 locales (06H21 GMT) desde la Base de la Fuerza Espacial Vandenberg en California."Todos los sistemas, así como el clima, lucen bien para el lanzamiento del Falcon 9 esta noche", tuiteó la compañía de Elon Musk.Su objetivo es Dimorphos, una "luna" de unos 160 metros (dos estatuas de la libertad) de ancho, que rodea a un asteroide mucho

Vendido por 11,6 millones de euros un borrador de la Teoría de la Relatividad de Albert Einstein
######
Christie's asegura que el manuscrito de 54 páginas del físico alemán y el ingeniero suizo Michele Besso es "excepcional", puesto que normalmente destruía sus documentos de trabajo
######
Un borrador del trabajo manuscrito sobre la Teoría de la Relatividad general del científico alemán Albert Einstein fue adjudicado este miércoles en París por la cifra récord de 11,6 millones de euros (unos 13,1 millones de dólares).Este manuscrito de 54 páginas perteneciente al físico alemán y a su amigo e ingeniero suizo Michele Besso es "excepcional", puesto que normalmente Einstein destruía los documentos de trabajo de sus teorías, según contó a Efe el especialista de la casa de subastas Christie's en libros y manuscritos, Adrien Legendre.El documento, inicialmente valorado entre 2 y 3 millones de euros, muestra "una fase crucial en el desarrollo de la Teoría de la Relatividad general" y es "el ma

Volcán La Palma, última hora en directo | Aumenta la magnitud de la erupción del nivel dos al tres por la elevada emisión de piroclastos
######
Ha superado la emisión de 10 millones de metros cúbicos de piroclastos
######
El volcán de La Palma, que comenzó su erupción hace 61 días en Cumbre Vieja, ha superado la emisión de 10 millones de metros cúbicos de piroclastos, por lo que el comité científico del Plan de emergencias volcánicas de Canarias (Pevolca) ha elevado su índice de explosividad de 2 a 3, en una escala de 8.La escala se calcula fundamentalmente por la cantidad de material piroclástico emitido y en este caso se ha elevado el nivel al haberse superado los 10 millones de metros cúbicos, no porque haya habido cambios ni en el mecanismo eruptivo ni en la explosividad, subrayó la portavoz del comité científico, Carmen López, tras la reunión diaria del Pevolca. De esta manera, el volcán no ha cambiado, mantiene el mismo mecanismo eruptivo fisular, estromboliano con pulsos retroma

...

Julio A., primer muerto en el volcán de La Palma: un carretillero de 72 años que quitaba la ceniza de su casa
######
Fuentes de la familia confirman que su cadáver fue encontrado en el interior del domicilio, en la zona de exclusión. A la espera de la autopsia, se desconoce si su muerte se debió a causas naturales o por los gases tóxicos de la erupción
######
Se llamaba Julio A.C., tenía 72 años, había trabajado como carretillero y con un elevador en una empresa llamada Ruiz Romero, y ayer había entrado en Las Manchas, justo debajo del volcán Cabeza de Vaca, a quitar ceniza del techo de su casa.Cuando la cuadrilla que le acompañaba salió del lugar, Julio, por motivos no aclarados, se quedó en el lugar, muy probablemente para quitar la ceniza de la casa de su hermana Irene, fallecida hace varios años.Un par de horas más tarde, hacia las 17 horas, personas cercanas se dieron cuenta de que no había salido de la zona de exclusión. Habrá que aclarar este extremo, puesto que las personas que

Registran un terremoto de magnitud 2,2 con epicentro en Ogíjares
######
El seísmo ha tenido lugar a las 5:45 de este sábado, con una profundidad de seis kilómetros
######
El Instituto Geográfico Nacional (IGN) ha registrado esta madrugada un terremoto de magnitud 2,2 con epicentro en el municipio de Ogíjares, próximo a Granada capital, que ha podido ser sentido por la población.Según ha informado el organismo dependiente del Ministerio de Transportes, Movilidad y Agenda Urbana, el seísmo ha tenido lugar a las 5:45 de este sábado, con una profundidad de seis kilómetros. Algunos vecinos de la zona han publicado en redes sociales como Twitter comentarios sobre este temblor y han indicado que han llegado a sentir el terremoto y cómo "crujían" sus viviendas.
######
Terremotos|Ciencia y Salud|ciencia|Andalucía|Granada|


Muere Winter, la delfín sin cola protagonista de la conmovedora película Dolphin Tale
######
Winter saltó a la fama con su conmovedora historia llevada al cine en la películ

Un grupo de niños descubre en Nueva Zelanda el fósil de una nueva especie de pingüinos gigantes
######
Estos pingüinos medían 1,38 metros de alto, tenían las patas especialmente largas y se cree que vivieron hace más de 27 millones de años
######
Corría el año 2006 cuando un grupo de niños, mientras buscaban fósiles de crustáceos en un campamento de verano en Nueva Zelanda, encontraron un fósil muy diferente. Se trataba del esqueleto de un pingüino gigante desconocido hasta la fecha. Así lo han determinado los investigadores de la Massey University de Nueva Zelanda y el Museo Bruce, en Estados Unidos, tras analizar el fósil con un escáner 3D. La investigación, que se ha publicado en la revista 'Journal of Vertebrate Paleontology', define a esta nueva especie como Kairuku waewaeroa y la califica como "una nueva especie de pingüinos gigantes".Se cree que este pingüino puede tener entre 27,3 y 34,6 millones de años y su altura alcanzaba los 1,38 metros. Su nombre, 'waewaeroa', significa "

La fatiga volcánica pasa factura: "Duermen mal, comen mal... no ven salida"
######
Los expertos advierten de la "dificultad añadida" de enfrentarse a una crisis que no parece tener fin. "El agente estresor sigue ahí, no te puedes relajar"
######
Se acerca a su octava semana el volcán, que sigue expulsando lava y manchando el cielo de ceniza como el primer día. El 19 de septiembre, no se podía descartar que fuera una crisis corta, que cesara en cualquier momento. Hoy, nadie sabe cuánto tiempo
######
Noticia sin etiquetas


Volcán La Palma, última hora en directo | El Instituto Volcanológico avisa de "un cambio en la dinámica" de la erupción
######
Los expertos de ese organismo basan su pronóstico en la aparición de depósitos de azufre elemental en el cono del volcán de La Palma
######
El Instituto Volcanológico de Canarias ha asegurado en la tarde de este viernes que se está produciendo "un cambio claro en la dinámica" del volcán de La Palma. El mensaje, difundido en su cuenta de Twitte

Reino Unido pide perdón a una ministra israelí con discapacidad por no haber podido acceder a la Cumbre del Clima con su silla de ruedas
######
Es triste que la ONU, que promueve la accesibilidad para las personas con discapacidad, en 2021 no brinde accesibilidad a sus eventos", lamentó en su cuenta. 
######
El Reino Unido se ha disculpado con la ministra israelí de Energía, Karine Elharrar, que tiene una discapacidad, porque ayer no pudo asistir a las reuniones de la cumbre del clima COP26 al no estar los accesos al recinto habilitados para sillas de ruedas."Pido disculpas profunda y sinceramente a la ministra. Queremos una cumbre de la COP26 acogedora e inclusiva para todos", escribió en su cuenta de Twitter el embajador británico en Israel, Neil Wigan. Elharrar, que sufre distrofia muscular, viajó el domingo a Glasgow junto al primer ministro, Naftali Benet, para participar este lunes en las reuniones de la COP26, a cuyo recinto no pudo acceder después de dos horas de intentos de lo

In [70]:
getFieldsFromHref(url_tecnologia)

Facebook acaba con una red de acoso fundada por antivacunas y que atacaba a médicos
######
A su vez, han eliminado una red dedicada a la denuncia masiva de perfiles con el fin de bloquearlos
######
Meta ha eliminado de Facebook e Instagram una red por acoso y hostigamiento masivo, identificada en Italia y Francia, otra por denuncias falsas masivas en Vietnam y cuatro por comportamiento no auténtico coordinado dirigidas contra usuarios de distintos países. Las redes sobre las que ha actualizado la compañía plantean un reto a la hora de combatirlas ya que sus acciones no acaban de encajar en las políticas de la compañía o solo violan una a la vez, como explica en el informe de amenazas que, por primera vez, reúne distintos interrupciones, como son el comportamiento no auténtico coordinado y dos nuevos protocolos: acoso y denuncia falsa masivos. Meta ha eliminado una red de cuentas que actuaba de forma coordinada para hostigar y acosar a médicos, periodistas y políticos. Identificada en I

Una coalición de empresas europeas denuncia a Microsoft por la integración de OneDrive y Teams
######
La coalición ha denunciado que Microsoft "presiona agresivamente a los consumidores para que se registren y entreguen sus datos"
######
Una coalición de empresas de 'software' y 'cloud' de la Unión Europea se ha sumado a una denuncia presentada por Nextcloud contra Microsoft en la Comisión Europea por considerar que la integración de aplicaciones como OneDrive y Teams en los sistemas operativos Windows resulta anticompetitiva para los desarrolladores independientes.En los últimos años, gigantes tecnológicos como Microsoft, Google y Amazon han aumentado su cuota de mercado de 'software' como servicio (SaaS, por sus siglas en inglés) hasta el 66 por ciento en Europa, mientras que los proveedores locales oscilan entre el 26 y el 16 por ciento.La coalición ha denunciado que Microsoft integra servicios como OneDrive y Teams en su sistema operativo Windows y con ello "presiona agresivamente 

Cybermonday 2021: ofertas y descuentos de última hora en tecnología
######
Ya se acaban las ofertas del Black Friday y el Cybermonday, pero hay buenos descuentos y los habrá hasta la última hora de hoy
######
El Cybermonday llega a su fin. Aunque el Black Friday ha durado más de una semana, las ofertas centradas en tecnología son algo más breves. Pero no por ello peores, precisamente. Sigue habiendo buenas ofertas del Cybermonday en móviles, monitores, televisores e incluso en electrodomésticos, y también en todo lo que rodead a estos productos, como pilas, cargadores, soportes...Como siempre, os pedimos un poquito de paciencia con los productos listados aquí abajo. Es posible que la oferta haya terminado o que el precio se haya alterado. Estamos actualizando constantemente, pero siempre se nos escapa algún error.Paquete de Office completo que incluye Microsoft Word, Excel, OneNote, Outlook, Access, PowerPoint, Publisher y Skype. Rebajado a 20,90 euros.Unos auriculares Bluetooth al día

Computación cuántica, la utopía (cada vez más cercana) que revolucionará la informática
######
Juan Ignacio Cirac, uno de los grandes precursores de esta especialidad de la física, explica los últimos avances en la construcción de este nuevo tipo de ordenadores, que serán más rápidos y potentes
######
Juan Ignacio Cirac es de esos investigadores a los que les suele acompañar el sobrenombre de uno de los padres de... seguido de algún campo de la ciencia. En su caso concreto, por ser el ideólogo de la computación cuántica. Una de las grandes utopías de este campo de la física durante décadas que, gracias a él, ya ha dejado de serlo, y que promete la aparición de ordenadores capaces de resolver problemas y tareas impensables para la tecnología actual.Eso, precisamente, ha llevado a este manresano a ser no sólo uno de los científicos más citados, sino también de los más galardonados. Un Premio Príncipe de Asturias, la Medalla Max Planck o el Premio Wolf (el más prestigioso después del Nobe

Cyber Monday 2021: estos son los mejores chollos en tecnología que puedes comprar en Amazon
######
El Black Friday ya ha pasado. Sin embargo, las grandes ofertas en tecnología aun duran hasta mañana lunes 29 de noviembre
######
Aunque el Black Friday 2021 ya ha llegado a su fin, todavía estás a tiempo de disfrutar de las grandes ofertas de tecnología que ofrece este Cyber Monday. Aunque el Cyber Monday se celebra oficialmente este lunes 29 de noviembre, las ofertas se han adelantado al fin de semana ya es posible echarle el guanta a ofertas y chollos de la mejor tecnología. Por eso, en PIXEL te presentamos esta selección de artículos que más merecen la pena sobre este Cyber Monday 2021.Echo Dot (3.ª generación) - Altavoz inteligente con Alexa, tela de color antracita. Rebajado de 49,99 a 19,99 euros.LE Linterna LED Alta Potencia, Linterna de Mano Brillante L8000, Foco ajustable, Mini Linternas Antorcha Impermeable, Pilas AAA Incluidas para Ciclismo, Camping, Montañismo [Clase de eficie

...

El trabajo que trae la digitalización: ¿'forward' o un simple 'review'?
######
Noticia sin entradilla
######
La pandemia trajo uno de sus principales vuelcos al trabajo. Se llevó por delante el ámbito laboral tal y como lo conocíamos e instauró la digitalización a toda prisa. Con ella aparecieron muchos retos; otros ya estaban ahí antes del Covid-19. Pero casi todos siguen abiertos.Reiniciada la maquinaria proveyendo a los trabajadores del material y las conexiones imprescindibles, a las empresas aún les queda mucho trabajo para alcanzar los desafíos que presenta la organización de una estructura laboral digitalizada. «Los responsables de personas se han enfrentado a la gestión de crisis, el manejo de riesgos y la reasignación de la fuerza laboral al trabajo, en remoto», expone el director de personas y cultura de la empresa de workplace y facility management ISS España, Ricard Casas. Pero, ahora hay que escoger entre recuperar la normalidad laboral del pasado, perpetuando los retos qu

El creciente negocio de digitalizar a otros
######
Márketing digital, posicionamiento en internet, servicios en la nube, 'chat bots'... En los últimos años han proliferado todo tipo de consultorías especializadas que ayudan a las empresas más pequeñas en su transformación
######
Las compañías que ayudan a otras a entrar en la nueva forma de hacer negocios han proliferado en los últimos años y se han convertido en fundamentales para multitud de pequeñas empresas que buscaban su rumbo ante el cambio insoslayable de la transformación digital. Pasado el mal trago de una digitalización obligada, rápida y en algunos casos sin ningún tipo de planificación, muchas empresas se han visto en la tesitura de emprender algo para lo que necesitan asesoramiento, estrategia o formación, además de una preparación tecnológica definida.Existir en el ámbito digital requiere de apoyos y desarrollos específicos. Marketing digital y publicidad son los eslabones más visibles de una cadena que empieza dotándose

Computación cuántica: la tecnología llamada a cambiar el rumbo de la industria
######
Los ordenadores cuánticos son ya una nueva carrera tecnológica por la que pujan las 'Big Tech'. Pese a estar aún en ciernes, sus últimos avances ofrecen capacidades impensables para la computación tradicional
######
Casi cuatro décadas después de que el prestigioso físico Richard Feynman propusiera crear ordenadores que funcionasen según la física cuántica, esta idea acogida por mucho tiempo con visos de ciencia ficción ha pasado a ser una realidad tangible en los últimos años. La tecnología cuántica supone un cambio de paradigma que implica tener una visión empresarial frente a un futuro en el cual aún se desconoce su verdadero grado de desarrollo, pero en el que la industria no puede permitirse no formar parte de ello.Poco a poco, la participación de gigantes tecnológicos como Google, IBM y Amazon nos ha acercado más que nunca a superar el desafío de Feynman. Sobre todo, con pequeños y progresivos h

'Big data', redes sociales o ciberseguridad: continúa creciendo el interés de las empresas en perfiles tecnológicos
######
Las ofertas de empleo para profesionales con competencias digitales crecen mucho más rápido que el número de candidatos. Los especialistas en estas áreas son los más demandados
######
La pandemia ha acelerado una transición digital que tarde o temprano había que asumir pero que, aunque era previsible, no se esperaba que se hiciera de una forma tan precipitada. Esto ha cambiado radicalmente el mundo laboral en todos los sectores, dando un mayor peso a la tecnología y obligando a todos a adaptarse a un nuevo ecosistema que implica mucho más que la implantación del teletrabajo.No es casualidad que tras los meses más duros de la crisis sanitaria, el tejido empresarial esté más necesitado que nunca de personas con competencias digitales. Según datos de Mobile World Capital Barcelona, el número de ofertas de perfiles digitales creció en 2020 un 80% respecto al año anteri

Asus VivoBook Pro 14X: pantalla, pantalla y pantalla
######
El portátil de Asus no llega a ultrabook, ni falta que le hace. Se trata de un equipo con personalidad y pantalla despampanante que, eso sí, supera los 1.000 euros
######
Asus ha puesto en el mercado una navaja suiza en el mercado con su VivoBook Pro 14X, un portátil competente y capaz de hacer casi todo -no está enfocado al juego, aunque se defiende- que destaca sobre todo por una pantalla espectacular. Complementa este sobresaliente con varios notables que hacen de él una opción interesante y polifacética.El Vivobook Pro 14X OLED parte de los 1.199 euros, así que entra en un terreno complicado con mucha competencia y, sobre todo, ya en el segmento por el que se pasea el MacBook, con todo lo que ello conlleva. A pesar de ello, tiene poco que temer y bastante que aportar: no es perfecto, pero justifica su precio.A nivel de rendimiento, hay que tener en cuenta que no es un ordenador para jugar -se puede hacer y se defiende, per